In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset   
import torch.nn.functional as F
from torch.autograd import Variable

#构造训练集
#100张28*28的图片
x = torch.rand(100,28,28)# 均匀分布采样
#print(x)
y = torch.randn(100,28,28)#标准正态分布采样
#print(y)
x = torch.cat((x,y),dim=0)#合并数据集 torch.cat()
label =[1] *100 + [0]*100  
label = torch.tensor(label,dtype=torch.long)#label转化为张量

#设置网络结构多层感知机
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.fc1  = nn.Linear(28*28, 120) 
        self.fc2  = nn.Linear(120, 84)
        self.fc3  = nn.Linear(84, 2)
  
    def forward(self, x):
        x = x.view(-1, self.num_flat_features(x))#展平x [100,28,28]→[100,784]  -1自动计算batch_size
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = self.fc3(x) 
        return x
    def num_flat_features(self, x):
        size = x.size()[1:] 
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

#训练集处理
class TraindataSet(Dataset):
    def __init__(self,train_features,train_labels):
        self.x_data = train_features
        self.y_data = train_labels
        self.len = len(train_labels)
    
    def __getitem__(self,index):#根据索引返回单个数据样本
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.len

#设置损失函数
loss_func = nn.CrossEntropyLoss()

#设置K折划分
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)  #slice(start,end,step)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
    
    return X_train, y_train, X_valid,y_valid

def k_fold(k, X_train, y_train, num_epochs=3,learning_rate=0.001, weight_decay=0.1, batch_size=5):
    train_loss_sum, valid_loss_sum = 0, 0
    train_acc_sum ,valid_acc_sum = 0,0
    
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net =  Net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size) 
       
        print('*'*10,'第',i+1,'折','*'*10)
        print('训练集损失:%.6f'%train_ls[-1][0],'训练集准确度:%.4f'%valid_ls[-1][1],\
              '测试集损失:%.6f'%valid_ls[-1][0],'测试集准确度:%.4f'%valid_ls[-1][1])
        train_loss_sum += train_ls[-1][0]
        valid_loss_sum += valid_ls[-1][0]
        train_acc_sum += train_ls[-1][1]
        valid_acc_sum += valid_ls[-1][1]
    print('#'*5,'最终k折交叉验证结果','#'*5)
    print('训练集累积损失:%.4f'%(train_loss_sum/k),'训练集累积准确度:%.4f'%(train_acc_sum/k),\
          '测试集累积损失:%.4f'%(valid_loss_sum/k),'测试集累积准确度:%.4f'%(valid_acc_sum/k))

#设置训练函数
def train(net, train_features, train_labels, test_features, test_labels, num_epochs, learning_rate,weight_decay, batch_size):
    train_ls, test_ls = [], []
    dataset = TraindataSet(train_features, train_labels) 
    train_iter = DataLoader(dataset, batch_size, shuffle=True)
    
    optimizer = torch.optim.Adam(params=net.parameters(), lr= learning_rate, weight_decay=weight_decay)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            output  = net(X)
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
       
        train_ls.append(log_rmse(0,net, train_features, train_labels)) 
        if test_labels is not None:
            test_ls.append(log_rmse(1,net, test_features, test_labels))

    return train_ls, test_ls

def log_rmse(flag,net,x,y):
    if flag == 1:
        net.eval()
    output = net(x)
    result = torch.max(output,1)[1].view(y.size())
    corrects = (result.data == y.data).sum().item()
    accuracy = corrects*100.0/len(y)
    loss = loss_func(output,y)
    net.train()
    
    return (loss.data.item(),accuracy)

#调用交叉验证函数
k_fold(10,x,label)
#k划分训练包括 划分 数据集准备 训练函数

********** 第 1 折 **********
训练集损失:0.041464 训练集准确度:100.0000 测试集损失:0.030746 测试集准确度:100.0000
********** 第 2 折 **********
训练集损失:0.038620 训练集准确度:100.0000 测试集损失:0.024316 测试集准确度:100.0000
********** 第 3 折 **********
训练集损失:0.042163 训练集准确度:100.0000 测试集损失:0.026072 测试集准确度:100.0000
********** 第 4 折 **********
训练集损失:0.040669 训练集准确度:100.0000 测试集损失:0.037004 测试集准确度:100.0000
********** 第 5 折 **********
训练集损失:0.035451 训练集准确度:100.0000 测试集损失:0.024351 测试集准确度:100.0000
********** 第 6 折 **********
训练集损失:0.036925 训练集准确度:100.0000 测试集损失:0.386534 测试集准确度:100.0000
********** 第 7 折 **********
训练集损失:0.040762 训练集准确度:95.0000 测试集损失:0.408720 测试集准确度:95.0000
********** 第 8 折 **********
训练集损失:0.037554 训练集准确度:100.0000 测试集损失:0.323398 测试集准确度:100.0000
********** 第 9 折 **********
训练集损失:0.038682 训练集准确度:100.0000 测试集损失:0.420865 测试集准确度:100.0000
********** 第 10 折 **********
训练集损失:0.038031 训练集准确度:95.0000 测试集损失:0.403737 测试集准确度:95.0000
##### 最终k折交叉验证结果 #####
训练集累积损失:0.0390 训练集累积准确度:100.0000 测试集累积损失:0.2086 测试集累积准确度:99.0000
